# Gerenciamento de tráfego - parte 2

Nesta seção iremos abordar mais formas de gerenciar o tráfego com o Istio, até o momento vimos basicamente uma formas, o roteamento por nome do host (FQDN).

Nosso novo cenário a nossa aplicação irá expor os serviços de _login_ e _order_ e algumas das suas APIs para outras aplicações.

Não precisamos expor cada um dos serviços da nossa aplicação, podemos criar um ou mais _VirtualServices_ que representem esse cenário:

| url interna               | url externa                                | service     | port |
| ---                       | ---                                        | ---         | ---  |
|                           | https://www.simul-shop.com                 | front-end   | 8000 |
|                           | https://www.simul-shop.com/login           | login       | 8000 |
|                           | https://www.simul-shop.com/catalogue       | catalogue   | 8000 |
| http://orders             | https://www.simul-shop.com/order           | orders      | 8000 |
| http://shipping           | https://www.simul-shop.com/shipping        | shipping    | 8000 |
| http://cart               | https://www.simul-shop.com/cart            | cart        | 8000 |
| http://payment            | https://www.simul-shop.com/payment         | payment     | 8000 |
| http://accounts           | https://www.simul-shop.com/accounts        | accounts    | 8000 |
| http://orders/db          |                                            | orders-db   | 8000 |
| http://queue              |                                            | queue       | 8000 |
| http://cart/db            |                                            | cart-db     | 8000 |
| http://accounts/db        |                                            | accounts-db | 8000 |

Mantenha um terminal aberto e o Kiali, iremos utilizá-los com frequência.

Vamos determinar a URI do Ingress Gateway e configurá-lo:

In [51]:
# Configurando acesso ao Ingress
export INGRESS_HOST=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
export INGRESS_PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="http2")].port}')
export SECURE_INGRESS_PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="https")].port}')
export TCP_INGRESS_PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="tcp")].port}')

echo "Ingress uri"
echo http://$INGRESS_HOST:$INGRESS_PORT
echo https://$INGRESS_HOST:$SECURE_INGRESS_PORT

Ingress uri
http://20.62.230.230:80
https://20.62.230.230:443


In [3]:
# Configurar um ingress gateway
kubectl apply -f exemplos/simul-shop/istio/10/default-gateway.yaml

gateway.networking.istio.io/default-gateway created


### Discussão: Versionamento de recursos web

Embora esteja fora do escopo desse curso, essa é uma discussão que em algum momento do desenvolvimento do sistema você terá que enfrentar, como indicar para os consumidores a versão da sua aplicação.

Podemos separar o tema em duas partes:

* Interface com usuário final
* APIs


#### Interface web

Não é interessante apresentar qualquer forma de versionamento para o usuário final, ou seja, não deveríamos expor qustões internas do sistemas, como portas, protocolos e versões para os usuários finais.

Também dificultaria testes A/B, implantações canário, ou qualquer forma de manipulação das requisições que necessitasse de transparência para o consumidor.

#### APIs

É uma estória completamente diferente, os consumidores destes recursos procuram algum tipo de garantia de que as versões que utilizam são estáveis, que o contrato que estabeleceram com o produtor não seja alterado para aquele ponto no tempo.

Existem prós e contras para cada escolha no forma de versionamento, entre elas:
    
* **Controle de versão por meio de caminho na URI**: http://www.simul-shop.com/orders/api/v1
    * Prós: os clientes podem armazenar recursos em cache facilmente
    * Contras: esta solução tem ramificações na base de código
* **Controle de versão por meio de parâmetros de consulta**: http://www.simul-shop.com/orders/api?version=v1
    * Prós: é uma maneira direta de criar uma versão de API e é fácil padronizar para a versão mais recente
    * Contras: os parâmetros de consulta são mais difíceis de usar para rotear solicitações para a versão adequada da API
* **Controle de versão por meio de cabeçalhos personalizados**: `curl -H “Accept-versions: v1” http://www.simul-shop.com/orders/api`
    * Prós: não confunde o URI com informações de versão
    * Contras: requer cabeçalhos personalizados
* **Controle de versão por meio de negociação de conteúdo**: `curl -H “Accept: application/vnd.xm.device+json; version=v1” http://www.simul-shop.com/orders/api`
    * Prós: nos permite criar versões de uma representação de recurso único em vez de criar versões de toda a API, o que nos dá um controle mais granular sobre as versões. Não requer a implementação de regras de roteamento de URI.
    * Contras: Exigir cabeçalhos HTTP com tipos de mídia torna mais difícil testar e explorar a API usando um navegador.

Do ponto de vista do design da APIs, incorporar a versão no URL não é uma prática recomendada porque resulta mudanças nas URLs, mesmo que o próprio recurso não tenha mudado.

[Roy Fielding](https://en.wikipedia.org/wiki/Roy_Fielding), um dos principais contribuidores para o protocolo HTTP e criador to REST, não recomenta utilizar qualquer forma de controle de versão para APIs ([Apresentação](https://www.slideshare.net/evolve_conference/201308-fielding-evolve)).

#### Por que não versionamento semântico?

De fato, a maioria das APIs utiliza o [versionamento semântico](https://semver.org/), mas não a versão completamente, pelo menos não na URI. Nada impediria que suas URL fossem `https://www.simul-shop.com/cart/api/1.0.2', porém o número da versão principal é o que você precisa para indicar aos seus consumidores a compatibilidade entre as versões. Os números que representam as versões menores e as correções não deveriam quebrar a compatibilidade e não justificariam a mudança da URL.

Se você precisa expressar a versão completa para seus consumidores, pode utilizar os cabeçalhos personalizados, como por exemplo o `Accept-versions` ou `Content-versions`.

#### Considerações

Em resumo, quando tratamos de interface web, geralmente procuramos ser o mais transparente o possível para o usuário final, mas se a questão é APIs, nossos consumidores, geralmente outros técnicos, necessitam de alguma garantia que as mesmas versões geram os mesmos resultados. Isso pode dificultar o uso de implementação canário por uma questão de semântica.

Felizmente, o Istio fornece recursos para lidar com todas as opções, então nós optamos por indicar a versão no cabeçalho, quando necessário.

Também optamos por não adicionar a palavra `api` na URL, por uma simples questão, ela não contribui para os nossos exemplos e você pode optar em criar seus serviços com interfaces web (micro front-ends) e programáticas (APIs), sendo assim, você poderia adicionar a palavra depois do roteamento do Istio, mas isso é com você.

## Rota baseada no caminho

![rota baseada no caminha](media/path-based-routes.png)

Vamos implementar a tabela acima, já vimos como configurar _VirtualService_ para hosts, vamos configurá-lo para caminho, a estrutura para as URIs do front-end

| url interna               | url externa                                | service     | port |
| ---                       | ---                                        | ---         | ---  |
|                           | https://www.simul-shop.com                 | front-end   | 8000 |
|                           | https://www.simul-shop.com/login           | login       | 8000 |
|                           | https://www.simul-shop.com/catalogue       | catalogue   | 8000 |


```yaml
spec:
  hosts:
    - "*"
  gateways:
  - default-gateway
  http:
  - match: # <-- Regra prefixo /catalogue
    - uri:
        prefix: /login
    rewrite:
      uri: /
    route:
    - destination:
        host: login # FQDN completo ou abreviado
        subset: v1
  - match: # <-- Regra prefixo /catalogue
    - uri:
        prefix: /catalogue
    rewrite:
      uri: /
    route:
    - destination:
        host: catalogue # FQDN completo ou abreviado
        subset: v1
  - route: # <-- Rota padrão
    - destination:
        host: front-end 
        subset: v1
```

In [52]:
# Destination rules
kubectl apply -f exemplos/simul-shop/istio/11/simul-shop-route-path-dr.yaml

destinationrule.networking.istio.io/front-end-drs configured
destinationrule.networking.istio.io/login-drs created
destinationrule.networking.istio.io/catalogue-drs created


In [56]:
# VirtualService
kubectl apply -f exemplos/simul-shop/istio/11/simul-shop-route-path.yaml

virtualservice.networking.istio.io/simulshop-routes created


Testando as regras:

In [59]:
# front-end
http -v "$INGRESS_HOST:$INGRESS_PORT/"

GET / HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0



HTTP/1.1 200 OK
content-length: 108
content-type: application/json
date: Tue, 22 Dec 2020 16:52:30 GMT
server: istio-envoy
x-envoy-upstream-service-time: 5

{
    "app": "front-end",
    "description": "Hi there!",
    "name": "greetings",
    "version": "v1",
    "when": "2020-12-22 16:52:30"
}




In [57]:
# Login
http -v "$INGRESS_HOST:$INGRESS_PORT/login"

GET /login HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0



HTTP/1.1 200 OK
content-length: 104
content-type: application/json
date: Tue, 22 Dec 2020 16:49:19 GMT
server: istio-envoy
x-envoy-upstream-service-time: 819

{
    "app": "login",
    "description": "Hi there!",
    "name": "greetings",
    "version": "v1",
    "when": "2020-12-22 16:49:24"
}




In [58]:
# Catalogue
http -v "$INGRESS_HOST:$INGRESS_PORT/catalogue"

GET /catalogue HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0



HTTP/1.1 200 OK
content-length: 108
content-type: application/json
date: Tue, 22 Dec 2020 16:49:30 GMT
server: istio-envoy
x-envoy-upstream-service-time: 773

{
    "app": "catalogue",
    "description": "Hi there!",
    "name": "greetings",
    "version": "v1",
    "when": "2020-12-22 16:49:30"
}




In [ ]:
Removendo as regras

In [65]:
# Destination rules
kubectl delete -f exemplos/simul-shop/istio/11/simul-shop-route-path-dr.yaml
# VirtualService
kubectl delete -f exemplos/simul-shop/istio/11/simul-shop-route-path.yaml

destinationrule.networking.istio.io "front-end-drs" deleted
destinationrule.networking.istio.io "login-drs" deleted
destinationrule.networking.istio.io "catalogue-drs" deleted
virtualservice.networking.istio.io "simulshop-routes" deleted


## Rota baseada no cabeçalho

Os _VirtualServices_ são uma das principais configurações do gerenciamento de tráfico do Istio, nesse exemplo iremos utilizar os dados do cabeçalho da requisição para decidir qual versão do serviço será servida para o requisitante.

Esse tipo de controle permite realizar testes e entregas para grupos de usuários. Diferente da entrega canário, que direciona o tráfego com base em percentuais, a rota baseada no cabeçalho, permite mirar grupos específicos.

Neste cenário os usuários que acessarem o front-end na região _Southeast_ serão direcionados para a versão 2 e os demais para a versão 1. 

Vamos aplica a configuração [front-end-route-header.yaml](exemplos/simul-shop/istio/11/route-header/front-end-route-header.yaml).

In [8]:
# Versão 2 do front-end
kubectl apply -f exemplos/simul-shop/manifests/8/front-end-deployment-v2.yaml

deployment.apps/front-end-v2 created


In [20]:
# Destination rules para o VirtualService
kubectl apply -f exemplos/simul-shop/istio/11/front-end-destination-rules.yaml

destinationrule.networking.istio.io/front-end-drs configured


In [13]:
# Configurar do VirtualService para rota baseada no cabeçalho
kubectl apply -f exemplos/simul-shop/istio/11/front-end-route-header.yaml

virtualservice.networking.istio.io/front-end configured


Nossa aplicação já pode ser acessada pela uri http://INGRESS_HOST:INGRESS_PORT/login

In [22]:
# Rota padrão
http -v "$INGRESS_HOST:$INGRESS_PORT"

GET / HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0



HTTP/1.1 200 OK
content-length: 108
content-type: application/json
date: Tue, 22 Dec 2020 15:11:09 GMT
server: istio-envoy
x-envoy-upstream-service-time: 2

{
    "app": "front-end",
    "description": "Hi there!",
    "name": "greetings",
    "version": "v1",
    "when": "2020-12-22 15:11:11"
}




A versão acessada foi a v1, vamos adicionar o cabeçalho.

In [23]:
# Regra #1 - campo user-region do cabeçalho igual a Southeast
http -v "$INGRESS_HOST:$INGRESS_PORT" "user-region: Southeast"

GET / HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0
user-region: Southeast



HTTP/1.1 200 OK
content-length: 108
content-type: application/json
date: Tue, 22 Dec 2020 15:11:20 GMT
server: istio-envoy
x-envoy-upstream-service-time: 1069

{
    "app": "front-end",
    "description": "Hi there!",
    "name": "greetings",
    "version": "v2",
    "when": "2020-12-22 15:11:24"
}




Você ppode usar uma combinação de campos do cabeçalho, campos [padrão](https://en.wikipedia.org/wiki/List_of_HTTP_header_fields), tais como:

* content-length
* content-type
* date
* Host
* User-Agent

Removendo as regras

In [67]:
# Destination rules para o VirtualService
kubectl delete -f exemplos/simul-shop/istio/11/front-end-destination-rules.yaml
# Configurar do VirtualService para rota baseada no cabeçalho
kubectl delete -f exemplos/simul-shop/istio/11/front-end-route-header.yaml

destinationrule.networking.istio.io "front-end-drs" deleted
virtualservice.networking.istio.io "front-end" deleted


## Multiplas regras de tráfego

As regras são avaliadas na ordem, Uma lista ordenada de regras de rota para tráfego HTTP. As rotas HTTP serão aplicadas às portas de serviço da plataforma chamadas 'http -' / 'http2 -' / 'grpc- *', portas de gateway com protocolo HTTP / HTTP2 / GRPC / TLS-terminated-HTTPS e portas de entrada de serviço usando HTTP / HTTP2 / Protocolos GRPC. A primeira regra que corresponde a uma solicitação de entrada é usada.

Neste cenário queremos combinar a uri com campos do cabeçalho, somente requisições com uri iniciando com `/front-end` e cabeçalho com o campo `user-region: Southeast` serão direcionados para a v2.

Após os testes, iremos direcionar as requisições com prefixo `/front-end` para a v2, mas durante os testes, serão direcionados para a v1.

E como boa prática, se nenhuma das regras for satisfeita, a requisição será direcionada para a v1.

In [68]:
# DestinationRules
kubectl apply -f exemplos/simul-shop/istio/11/front-end-destination-rules.yaml

destinationrule.networking.istio.io/front-end-drs created


In [69]:
# Virtual Service
kubectl apply -f exemplos/simul-shop/istio/11/front-end-multi-route.yaml

virtualservice.networking.istio.io/front-end created


In [70]:
# Rota padrão - v1
http -v "$INGRESS_HOST:$INGRESS_PORT/" "user-region: Southeast"

GET / HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0
user-region: Southeast



HTTP/1.1 200 OK
content-length: 108
content-type: application/json
date: Tue, 22 Dec 2020 17:30:57 GMT
server: istio-envoy
x-envoy-upstream-service-time: 12

{
    "app": "front-end",
    "description": "Hi there!",
    "name": "greetings",
    "version": "v1",
    "when": "2020-12-22 17:30:58"
}




In [71]:
# Rota padrão - v1
http -v "$INGRESS_HOST:$INGRESS_PORT/"

GET / HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0



HTTP/1.1 200 OK
content-length: 108
content-type: application/json
date: Tue, 22 Dec 2020 17:30:57 GMT
server: istio-envoy
x-envoy-upstream-service-time: 5

{
    "app": "front-end",
    "description": "Hi there!",
    "name": "greetings",
    "version": "v1",
    "when": "2020-12-22 17:31:03"
}




In [72]:
# Regra #2: uri iniciando em /front-end - v1 (mas pode ser alterada para outra versão)
http -v "$INGRESS_HOST:$INGRESS_PORT/front-end"

GET /front-end HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0



HTTP/1.1 200 OK
content-length: 108
content-type: application/json
date: Tue, 22 Dec 2020 17:30:57 GMT
server: istio-envoy
x-envoy-upstream-service-time: 2

{
    "app": "front-end",
    "description": "Hi there!",
    "name": "greetings",
    "version": "v1",
    "when": "2020-12-22 17:31:07"
}




In [73]:
# Regra #1: user-regio=Southeast e uri iniciando em /front-end - v2
http -v "$INGRESS_HOST:$INGRESS_PORT/front-end" "user-region: Southeast"

GET /front-end HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0
user-region: Southeast



HTTP/1.1 200 OK
content-length: 108
content-type: application/json
date: Tue, 22 Dec 2020 17:31:11 GMT
server: istio-envoy
x-envoy-upstream-service-time: 7

{
    "app": "front-end",
    "description": "Hi there!",
    "name": "greetings",
    "version": "v2",
    "when": "2020-12-22 17:31:11"
}




In [74]:
# Rota padrão, mas não tem re-escrita da URI - 404
http -v "$INGRESS_HOST:$INGRESS_PORT/anything"

GET /anything HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0



HTTP/1.1 404 Not Found
content-length: 22
content-type: application/json
date: Tue, 22 Dec 2020 17:30:57 GMT
server: istio-envoy
x-envoy-upstream-service-time: 2

{
    "detail": "Not Found"
}




Removendo as regras

In [75]:
# DestinationRules
kubectl delete -f exemplos/simul-shop/istio/11/front-end-destination-rules.yaml
# Virtual Service
kubectl delete -f exemplos/simul-shop/istio/11/front-end-multi-route.yaml

destinationrule.networking.istio.io "front-end-drs" deleted
virtualservice.networking.istio.io "front-end" deleted


## Modificando os cabeçalhos de resposta

Os _VirtualServices_ podem adicionar ou remover campos do cabeçalho.

Neste exemplo, queremos que as requisições que não são originadas de _Southeast_ tenham o campo do cabeçalho `user-region` com o valor `other`. Essa informação pode ser utilizada para regras em outros serviços, ou para fins de log.

In [78]:
# DestinationRules
kubectl apply -f exemplos/simul-shop/istio/11/front-end-destination-rules.yaml

destinationrule.networking.istio.io/front-end-drs created


In [79]:
# VirtualService
kubectl apply -f exemplos/simul-shop/istio/11/front-end-change-header.yaml

virtualservice.networking.istio.io/front-end unchanged


In [80]:
# Rota padrão - v1
http -v "$INGRESS_HOST:$INGRESS_PORT/" "user-region: Southeast"

GET / HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0
user-region: Southeast



HTTP/1.1 200 OK
content-length: 108
content-type: application/json
date: Tue, 22 Dec 2020 17:33:10 GMT
server: istio-envoy
x-envoy-upstream-service-time: 6

{
    "app": "front-end",
    "description": "Hi there!",
    "name": "greetings",
    "version": "v2",
    "when": "2020-12-22 17:33:16"
}




In [81]:
# Rota padrão - v1
http -v "$INGRESS_HOST:$INGRESS_PORT/"

GET / HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0



HTTP/1.1 200 OK
content-length: 108
content-type: application/json
date: Tue, 22 Dec 2020 17:33:20 GMT
server: istio-envoy
user-region: other
x-envoy-upstream-service-time: 12

{
    "app": "front-end",
    "description": "Hi there!",
    "name": "greetings",
    "version": "v1",
    "when": "2020-12-22 17:33:21"
}




Todas as requisições que forem direcionadas pela rota padrão terão o campo `user-region: other` adicionado.

In [ ]:
Removendo as regras:

In [82]:
# DestinationRules
kubectl delete -f exemplos/simul-shop/istio/11/front-end-destination-rules.yaml
# VirtualService
kubectl delete -f exemplos/simul-shop/istio/11/front-end-change-header.yaml

destinationrule.networking.istio.io "front-end-drs" deleted
virtualservice.networking.istio.io "front-end" deleted


## Retentativas e timeouts

Arquiteturas distribuídas significa solicitações na rede, aumentando a chance de falhas temporárias como congestionamento da rede.

Adicionar políticas de repetição para solicitações ajuda a construir resiliência em uma arquitetura de serviços. Freqüentemente, essa lógica de repetição é incorporada no serviço, mas com o Istio, você pode definir políticas de repetição com uma regra de tráfego, a serem executadas pelos sidecars, permitindo padronizar as políticas de forma independente dos  protocolos e linguagens de programação.

Simular falhas é difícil, mas usaremos um recurso do próprio Istio para simular uma falha no serviço de _order_ 

In [112]:
# Versão 1 do front-end
kubectl apply -f exemplos/simul-shop/manifests/11/front-end-deployment-error.yaml

deployment.apps/front-end-v1 created
deployment.apps/front-end-v2 created


In [114]:
# DestinationRules
kubectl apply -f exemplos/simul-shop/istio/11/front-end-destination-rules.yaml
kubectl apply -f exemplos/simul-shop/istio/11/login-destination-rules.yaml

destinationrule.networking.istio.io/front-end-drs unchanged
destinationrule.networking.istio.io/login-drs unchanged


Você deverá ver errors 500 no kiali e jaeger. No jaeger filtre os rastros colocando em _Service_ escolha `front-end.default` e em _Tags_ preencha com o valor `http.status_code=500`

In [127]:
# VirtualService
kubectl apply -f exemplos/simul-shop/istio/11/login-retry.yaml
kubectl apply -f exemplos/simul-shop/istio/11/front-end-vs.yaml

virtualservice.networking.istio.io/login created


Vamos testar:

In [128]:
# Gerando um pouco de tráfego
for i in $(seq 1 10);
    do http -v "$INGRESS_HOST:$INGRESS_PORT/s";
done

GET /s HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0



HTTP/1.1 200 OK
content-length: 100
content-type: application/json
date: Tue, 22 Dec 2020 22:02:03 GMT
server: istio-envoy
x-envoy-upstream-service-time: 3

{
    "app": "login",
    "description": "List ['']",
    "name": "split",
    "version": "v1",
    "when": "2020-12-22 22:02:04"
}


GET /s HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0



HTTP/1.1 200 OK
content-length: 100
content-type: application/json
date: Tue, 22 Dec 2020 22:02:03 GMT
server: istio-envoy
x-envoy-upstream-service-time: 2

{
    "app": "login",
    "description": "List ['']",
    "name": "split",
    "version": "v1",
    "when": "2020-12-22 22:02:04"
}


GET /s HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0



HTTP/1.1 200 OK
con

## Interrupção de circuíto

TODO

## Espelhando o tráfego

TODO